In [5]:
import time
import resource
import pandas as pd
import numpy as np

>Remember that dpkt opens only pcap and NOT pcapng

In [34]:
!capinfos -ts 'data/anon-Booter5.pcap'

File name:           data/anon-Booter5.pcap
File type:           Wireshark/tcpdump/... - pcap
File size:           305 MB


## Analysing dpkt 

In [22]:
import argparse
import dpkt
import socket
import os

In [30]:
time_custom4 = time.time()
mem_custom4 = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss 

inputfile = open('data/anon-Booter5.pcap')
pcapfile = dpkt.pcap.Reader(inputfile)

outputfile = open('data/anon-Booter5.txt','w')

for ts, buf in pcapfile:
    eth = dpkt.ethernet.Ethernet(buf)

    #FILTERING ONLY FOR IPv4 instead of packets ARP or IPv6
    if eth.type == dpkt.ethernet.ETH_TYPE_IP:
        ip = eth.data #Loading the content of the ethernet into a variable 'ip'
        
        timestamp = ts #1
        ip_ttl = ip.ttl #2
        
        ip_proto = ip.p #3
        sport = ""
        dport= ""
        tcp_flag =""
        http_data=""
        if (ip_proto != 6) and (ip_proto != 17): #It is not TCP or UDP
            continue
            
        ip_length = ip.len #4
        ip_src = socket.inet_ntoa(ip.src) #5
        ip_dst = socket.inet_ntoa(ip.dst) #6
        
        try: proto = ip.data #Loading the content of the 'ip' into a variable 'protocol' that can be for example ICMP, TCP, and UDP.
        except:
            continue
        
        sport = proto.sport #7
        dport = proto.dport #8


        if ip.p == 6 :
            try:
                tcp_flag += ("F" if (int( proto.flags & dpkt.tcp.TH_FIN ) != 0) else ".") #27
                tcp_flag += ("S" if (int( proto.flags & dpkt.tcp.TH_SYN ) != 0) else ".") #26
                tcp_flag += ("R" if (int( proto.flags & dpkt.tcp.TH_RST ) != 0) else ".") #25
                tcp_flag += ("P" if (int( proto.flags & dpkt.tcp.TH_PUSH) != 0) else ".") #24
                tcp_flag += ("A" if (int( proto.flags & dpkt.tcp.TH_ACK ) != 0) else ".") #23
                tcp_flag += ("U" if (int( proto.flags & dpkt.tcp.TH_URG ) != 0) else ".") #22
                tcp_flag += ("E" if (int( proto.flags & dpkt.tcp.TH_ECE ) != 0) else ".") #21
                tcp_flag += ("C" if (int( proto.flags & dpkt.tcp.TH_CWR ) != 0) else ".") #20
            except:
                print "EXCEPTION TCP FLAG"

            if proto.dport == 80:
                try: 
                    http = dpkt.http.Request(proto.data)
                    http_data = http.data
                except: continue
            
        fragments = 1 if (int(ip.off & dpkt.ip.IP_MF)!= 0) else 0  #8 This flag is set to a 1 for all fragments except the last one            

        print >> outputfile, str(timestamp)+';'+\
        str(ip_ttl)+';'+\
        str(ip_proto)+';'+\
        str(ip_length)+';'+\
        str(ip_src)+';'+\
        str(ip_dst)+';'+\
        str(sport)+';'+\
        str(dport)+';'+\
        str(tcp_flag)+';'+\
        str(fragments)+';'+\
        str(http_data)

mem_custom4 = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss -mem_custom4
time_custom4 = time.time() - time_custom4

print(time_custom4, mem_custom4)

(84.62819600105286, 8192)


### This is around two times more expensive than tcpdump with awk. However it is easier to create the output.